# Aggregate swaps

This notebook shows a method to get all Bancor trades over a time period.

Set the timestamps and sum the trade volume to get 24h volume, daily trading volume, etc.

This is not a suitable method over larger timeframes.


## Setup
---

In [1]:
from graphqlclient import GraphQLClient
import pandas as pd
import json
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
ENDPOINT = "https://api.thegraph.com/subgraphs/name/blocklytics/bancor"
client = GraphQLClient(ENDPOINT)

In [3]:
# Timestamps
start_time = int(datetime(2020,1,1).timestamp()) # Inclusive
end_time = int(datetime(2020,1,2).timestamp())   # Exclusive

## Fetch data
---

In [4]:
# Start with an empty df
df = pd.DataFrame()

# Results must be paginated
limit = 100
offset = 0
fetching_results = True

In [5]:
# Fetch paginated results
while fetching_results:
    QUERY = """
    {{
      swaps(
        first: {0}, 
        skip: {1},
        where:{{ 
          timestamp_gte:"{2}",
          timestamp_lt:"{3}"
        }},
        orderBy:timestamp,
        orderDirection:desc
      ) {{
        fromToken {{ id, symbol, decimals }}
        toToken {{ id, symbol, decimals }}
        amountPurchased
        amountReturned
        converterUsed {{ id }}
        timestamp
        transaction {{ id }}
      }}
    }}
    """.format(limit, offset, start_time, end_time)

    result = json.loads(client.execute(QUERY))
    _df = pd.DataFrame.from_records(json_normalize(result['data']['swaps']))
    
    # If df is empty, create it from new _df
    # Else append new _df to df
    if len(df.index) == 0:
        df = _df
    else:
        df = df.append(_df)
    
    # Prepare for pagination
    result_length = len(_df.index)
    if limit == result_length:
        offset += limit
    else:
        fetching_results = False

df.reset_index(drop=True, inplace=True)
df.sort_values('timestamp', ascending=False)
df.shape

(1979, 11)

# Format data
---
Convert wei values

In [6]:
df['amountPurchased_float'] = df.apply(lambda row: float(row['amountPurchased']) / 10 ** row['fromToken.decimals'], axis=1)
df['amountReturned_float'] = df.apply(lambda row: float(row['amountReturned']) / 10 ** row['toToken.decimals'], axis=1)

Get volume in base token terms (USDB or BNT)

In [7]:
df['volume_bnt'] = \
    df.apply(lambda row: row['amountPurchased_float'] if row['fromToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)\
    + df.apply(lambda row: row['amountReturned_float'] if row['toToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)

df['volume_usdb'] = \
    df.apply(lambda row: row['amountPurchased_float'] if row['fromToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\
    + df.apply(lambda row: row['amountReturned_float'] if row['fromToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\


Convert timestamp

In [8]:
df['timestamp'] = df.apply(lambda row: datetime.fromtimestamp(int(row['timestamp'])), axis=1)

In [9]:
df[['transaction.id', 'timestamp', 'amountPurchased_float', 'fromToken.symbol', 'amountReturned_float', 'toToken.symbol', 'volume_bnt', 'volume_usdb']]\
    .head(10)


,transaction.id,timestamp,amountPurchased_float,fromToken.symbol,amountReturned_float,toToken.symbol,volume_bnt,volume_usdb
0,0xd7fdecd22bbb4c3ea8d03c8d147fd80dbecf5ddd982b...,2020-01-01 23:58:52,0.325000,ETH,178.250420,BNT,178.250420,0.0
1,0xd7fdecd22bbb4c3ea8d03c8d147fd80dbecf5ddd982b...,2020-01-01 23:58:52,178.250420,BNT,10706.743161,ONG,178.250420,0.0
2,0xae5fdf5bbabc87c25053849c8204be7ea7d095c40980...,2020-01-01 23:51:49,129477.428571,XDCE,569.293187,BNT,569.293187,0.0
3,0xae5fdf5bbabc87c25053849c8204be7ea7d095c40980...,2020-01-01 23:51:49,569.293187,BNT,1.033888,ETH,569.293187,0.0
4,0xc2f5bc03455de9bbfab71a0485c5fbbfb30b6f4fce0d...,2020-01-01 23:51:34,1.000000,ETH,548.432656,BNT,548.432656,0.0
5,0xc2f5bc03455de9bbfab71a0485c5fbbfb30b6f4fce0d...,2020-01-01 23:51:34,548.432656,BNT,124200.849339,XDCE,548.432656,0.0
6,0xe523afe17f3677d346c02928ab85bcacf2239d66b081...,2020-01-01 23:35:09,0.999799,ETH,548.403194,BNT,548.403194,0.0
7,0xe523afe17f3677d346c02928ab85bcacf2239d66b081...,2020-01-01 23:35:09,548.403194,BNT,130.687893,None,548.403194,0.0
8,0xaaeed17cc86ff94f2342f2d12f07abe7c6e3552f1d48...,2020-01-01 23:32:34,730.000000,ENJ,246.882869,BNT,246.882869,0.0
9,0xaaeed17cc86ff94f2342f2d12f07abe7c6e3552f1d48...,2020-01-01 23:32:34,246.882869,BNT,0.448279,ETH,246.882869,0.0


# Volumes over period
---
## BNT Volume

In [10]:
df['volume_bnt'].sum()

3168037.6448270334

## USDB Volume

Avoid double-counting volume by excluding BNT volume already counted above.

In [11]:
# df['volume_usdb'].sum() # This will double-count BNT volume.

In [12]:
df[df['volume_bnt'] == 0]['volume_usdb'].sum()

5691.584718157279

## Trades not counted

These trades were not counted as they did not include a base token of BNT or USDB.

In [13]:
df[(df['volume_bnt'] == 0) & (df['volume_usdb'] == 0)]

,amountPurchased,amountReturned,converterUsed.id,fromToken.decimals,fromToken.id,fromToken.symbol,timestamp,toToken.decimals,toToken.id,toToken.symbol,transaction.id,amountPurchased_float,amountReturned_float,volume_bnt,volume_usdb
23,464000000000000000000,90930259354164736443066,0xaa8cec9cbd7d051ba86d9deff1ec0775bd4b13c5,18.0,0x0f9be347378a37ced33a13ae061175af07cc9868,AMNBNT,2020-01-01 23:05:18,18.0,0x737f98ac8ca59f2c68ad658e3c3d8c8963e40a4c,AMN,0xdde655ef629e07ca5adec27ef78865168e62195aac6b...,464.000000,90930.259354,0.0,0.0
24,26000000000000000000,26212567747392519044,0x06f7bf937dec0c413a2e0464bb300c4d464bb891,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2020-01-01 22:59:27,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0xfb28c875ba80c6c20f5245907e2e90bc39250751f6ce...,26.000000,26.212568,0.0,0.0
50,4004034,3991491574197289597,0x868229b43a8bcbdffb244dde874f52ade0b1c132,6.0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,2020-01-01 22:10:48,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x5920fd8ae8dc096420a1573e39226324add504392da8...,4.004034,3.991492,0.0,0.0
74,1000000000000000000000,9371728918029716582,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-01 21:52:34,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0xadd6eb95c76d8df28781cc849fb5b0b6e5f94aeb1ab3...,1000.000000,9.371729,0.0,0.0
84,190000000000000000000,2063050406584525357,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-01 21:43:27,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x1023006d4ec82988e42bf04d2b45ff1390efa286d221...,190.000000,2.063050,0.0,0.0
132,335000000000000000,170772005070210098619,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,2020-01-01 21:00:39,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,0xfb46b0e78474e4873cb129825994ce2493543b6f4121...,0.335000,170.772005,0.0,0.0
157,50608499367228790299,51052293708003933751,0x06f7bf937dec0c413a2e0464bb300c4d464bb891,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2020-01-01 20:36:10,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0xeeea0232ee7be4f7c353a9848ccd7ddc0d66e231853d...,50.608499,51.052294,0.0,0.0
170,186500000000000000,95079719294023979023,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,2020-01-01 20:12:09,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,0x780764a257d31fb0905c5c53eeaafa9e9ade73865b58...,0.186500,95.079719,0.0,0.0
198,2000000000000000000,1020002671416306829752,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,2020-01-01 19:38:08,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,0x152fab4108b19a513bed5a0f74b81baa0458cd4b71a5...,2.000000,1020.002671,0.0,0.0
207,419905275099429290000,2125170357090351895001,0x08b61ded2f558071fbdb827715e7aef16e76dd4f,18.0,0x0f2318565f1996cb1ed2f88e172135791bc1fcbf,ELFBNT,2020-01-01 19:26:42,18.0,0xbf2179859fc6d5bee9bf9158632dc51678a4100e,ELF,0x1ca8c3e783c58a36365ad12ab0ba068c2afe3620b73a...,419.905275,2125.170357,0.0,0.0


In [14]:
df.iloc[23]

amountPurchased                                      464000000000000000000
amountReturned                                     90930259354164736443066
converterUsed.id                0xaa8cec9cbd7d051ba86d9deff1ec0775bd4b13c5
fromToken.decimals                                                      18
fromToken.id                    0x0f9be347378a37ced33a13ae061175af07cc9868
fromToken.symbol                                                    AMNBNT
timestamp                                              2020-01-01 23:05:18
toToken.decimals                                                        18
toToken.id                      0x737f98ac8ca59f2c68ad658e3c3d8c8963e40a4c
toToken.symbol                                                         AMN
transaction.id           0xdde655ef629e07ca5adec27ef78865168e62195aac6b...
amountPurchased_float                                                  464
amountReturned_float                                               90930.3
volume_bnt               